In [1]:
import torch
x = torch.rand(5, 3)
print(x)


tensor([[0.2921, 0.2919, 0.1495],
        [0.8364, 0.3509, 0.9234],
        [0.0155, 0.5198, 0.8172],
        [0.2750, 0.9240, 0.7893],
        [0.0489, 0.3980, 0.4946]])
